Sources: https://stackoverflow.com/questions/6400416/figure-out-if-a-business-name-is-very-similar-to-another-one-python

In [1]:
import pandas as pd
import re
import os
from os.path import dirname, abspath
from IPython.display import display, HTML # Source: https://stackoverflow.com/questions/38783027/jupyter-notebook-display-two-pandas-tables-side-by-side

In [2]:
# Get Current Working Directory and Parent Path (for reading files in different folders)
## Source: https://stackoverflow.com/questions/30218802/get-parent-of-current-directory-from-python-script/30218825
d = dirname(dirname(abspath(os.getcwd())))
d

'C:\\Users\\Trang\\Desktop\\afn'

## Load Orbis Data & PPP Loan Data (Filtered)

Explore different cases of business names.

In [3]:
orbis_file = d+'/ppp-loan-data/Orbis database_six cities only.csv'
ppp_minority_file = d+'/ppp-loan-data/out/bay_bus_from_ppp_minority.csv'
ppp_file =d+'/ppp-loan-data/out/bay_bus_from_ppp.csv'

In [4]:
orbis_df = pd.read_csv(orbis_file)
orbis_df.head()

,Company name Latin alphabet,Inactive,Quoted,Branch,OwnData,Woco,Country ISO code,"NACE Rev. 2, core code (4 digits)",Consolidation code,Last avail. year,...,Latitude,Longitude,Website address,"NAICS 2017, core code (4 digits)","NAICS 2017, core code - description",Size classification,Type of entity,Ethnic minority owned indicator (in US),Woman owned indicator (in US),Study Area
0,CALIFORNIA NURSES ASSOCIATION,No,No,Yes,No,No,US,9412,LF,2019.0,...,"37° 48' 30.9"" N","122° 16' 1.2"" W",www.calnursesfoundation.org,8139,"Business, Professional, Labor, Political, and ...",Very large company,Corporate,No,No,Oakland
1,HONDA CARS OF CORONA,No,No,No,No,No,US,4519,LF,2019.0,...,"37° 49' 18.2"" N","122° 15' 37.2"" W",www.hondacarsofcorona.com,4411,Automobile Dealers,Large company,Corporate,No,No,Oakland
2,SIMI MANAGEMENT CORP,No,No,No,No,No,US,4519,LF,2019.0,...,"37° 49' 0.2"" N","122° 15' 48.6"" W",NaN,4411,Automobile Dealers,Large company,Corporate,No,No,Oakland
3,ECONOMY LUMBER CO OF OAKLAND,No,No,No,No,No,US,4778,LF,2019.0,...,"37° 46' 7.3"" N","122° 13' 10.9"" W",www.economylumberco.com,4441,Building Material and Supplies Dealers,Large company,Corporate,No,No,Oakland
4,BROADWAY FORD,No,No,No,No,No,US,4519,LF,2008.0,...,"37° 48' 51.1"" N","122° 15' 53.5"" W",www.mybroadwayford.com,4411,Automobile Dealers,Large company,Corporate,No,No,Oakland


In [5]:
orbis_df.columns

Index(['Company name Latin alphabet', 'Inactive', 'Quoted', 'Branch',
       'OwnData', 'Woco', 'Country ISO code',
       'NACE Rev. 2, core code (4 digits)', 'Consolidation code',
       'Last avail. year',
       'Operating revenue (Turnover)\nth USD Last avail. yr',
       'Number of employees\nLast avail. yr', 'Address line 1\nLatin Alphabet',
       'Address line 2\nLatin Alphabet', 'Postcode\nLatin Alphabet',
       'City\nLatin Alphabet', 'Region in country', 'Latitude', 'Longitude',
       'Website address', 'NAICS 2017, core code (4 digits)',
       'NAICS 2017, core code - description', 'Size classification',
       'Type of entity', 'Ethnic minority owned indicator (in US)',
       'Woman owned indicator (in US)', 'Study Area'],
      dtype='object')

In [5]:
ppp_df = pd.read_csv(ppp_file)
ppp_df.head()

,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,BorrowerZip,LoanStatusDate,...,NonProfit,ForgivenessAmount,ForgivenessDate,zip_code_5,Minority,YearApproved,NAICS_6,LongName,IndustrySubsector,NAICS_4
0,8585717704,05/01/2020,459.0,PPP,MARKET AUTO TRUCK COLLISION CORPORATION,140 SAN JOSE AVE,SAN JOSE,AL,95125,0,...,NaN,15281.29,05/13/2021,95125,Unanswered,2020,811121.0,"Automotive Body, Paint, and Interior Repair an...",Automotive Repair and Maintenance,8111.0
1,1464468601,03/13/2021,912.0,PPS,GOLD RUSH KETTLE KORN LLC,4690 E 2nd St Ste 9,Benicia,CA,94510-1008,0,...,NaN,0.00,NaN,94510,No,2021,722330.0,Mobile Food Services,Special Food Services,7223.0
2,2663308509,02/22/2021,912.0,PPS,KALIBER LABS INC,188 King St Unit 307,San Francisco,CA,94107-4903,0,...,NaN,0.00,NaN,94107,No,2021,541511.0,Custom Computer Programming Services,Computer Systems Design and Related Services,5415.0
3,4137348607,03/18/2021,912.0,PPS,ROSS MCDONALD COMPANY INC,1154 Stealth St,Livermore,CA,94551-9300,0,...,NaN,0.00,NaN,94551,Unanswered,2021,541420.0,Industrial Design Services,Specialized Design Services,5414.0
4,4315748602,03/18/2021,912.0,PPS,GATES EISENHART DAWSON,125 S Market St Ste 1200,San Jose,CA,95113-2288,0,...,NaN,0.00,NaN,95113,Unanswered,2021,541110.0,Offices of Lawyers,Legal Services,5411.0


In [6]:
ppp_minority_df = pd.read_csv(ppp_minority_file)
ppp_minority_df.head()

,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,BorrowerZip,LoanStatusDate,...,NonProfit,ForgivenessAmount,ForgivenessDate,zip_code_5,Minority,YearApproved,NAICS_6,LongName,IndustrySubsector,NAICS_4
0,5224738402,02/08/2021,912.0,PPS,ICHINA,70 Valley Ct,Atherton,CA,94027-6472,0,...,NaN,0.0,NaN,94027.0,Yes,2021,722511.0,Full-Service Restaurants,Restaurants and Other Eating Places,7225.0
1,5359188509,02/27/2021,912.0,PPS,BANZAI IZAKAYA INC.,1633 Bonanza St,Walnut Creek,CA,94596-4525,0,...,NaN,0.0,NaN,94596.0,Yes,2021,722511.0,Full-Service Restaurants,Restaurants and Other Eating Places,7225.0
2,5554978603,03/20/2021,912.0,PPS,RANGOON SUPER STARS LLC,542 Freya Way,Pleasant Hill,CA,94523-1712,0,...,NaN,0.0,NaN,94523.0,Yes,2021,722511.0,Full-Service Restaurants,Restaurants and Other Eating Places,7225.0
3,5681669004,05/22/2021,912.0,PPS,VIVA EL ESPANOL SPANISH SCHOOL INC.,3205 Stanley Blvd,Lafayette,CA,94549-3239,05/22/2021,...,NaN,0.0,NaN,94549.0,Yes,2021,624410.0,Child Day Care Services,Child Day Care Services,6244.0
4,9634738305,01/31/2021,912.0,PPS,JBR PARTNERS INC,1333 Evans Ave,San Francisco,CA,94124-1705,0,...,NaN,0.0,NaN,94124.0,Yes,2021,541820.0,Public Relations Agencies,"Advertising, Public Relations, and Related Ser...",5418.0


### Check Business Name and Address Columns

In [7]:
# All caps - matches PPP's format
display(orbis_df['Company name Latin alphabet'])
# All caps - need to standardize address (.upper() on PPP or .title() on Orbis)
display(orbis_df['Address line 1\nLatin Alphabet'])

0           CALIFORNIA NURSES ASSOCIATION
1                    HONDA CARS OF CORONA
2                    SIMI MANAGEMENT CORP
3            ECONOMY LUMBER CO OF OAKLAND
4                           BROADWAY FORD
                       ...               
92295                             STANDBY
92296                          CAFE DOLCI
92297                          SOLO SALON
92298             SEVEN TREES BOTTLE SHOP
92299    TERIYAKI HOUSE/HOUSE OF THAI INC
Name: Company name Latin alphabet, Length: 92300, dtype: object

0         2000 FRANKLIN ST
1            3330 BROADWAY
2            2825 BROADWAY
3              750 HIGH ST
4          2560 WEBSTER ST
               ...        
92295     935 AIRPORT BLVD
92296       78 IDLEWOOD CT
92297         6 TUNITAS LN
92298    2164 GREENDALE DR
92299      243 RAINIER AVE
Name: Address line 1\nLatin Alphabet, Length: 92300, dtype: object

## Approach: Clean Names and Address Col to do String match

## Case Examples

Notes:
* Look at addresses to verify that business names correspond to the same business
* Options: NAICS (doesn't always match), ...

### 1. INC vs. INC.

Orbis: INDELICATO INC <br>
PPP: INDELICATO INC. <br>

**Solution:** Strip punctuation and match on just the letters

In [7]:
display(orbis_df[orbis_df['Company name Latin alphabet'] == 'INDELICATO INC'])
display(ppp_df[ppp_df['BorrowerName'] == 'INDELICATO INC.'])

,Company name Latin alphabet,Inactive,Quoted,Branch,OwnData,Woco,Country ISO code,"NACE Rev. 2, core code (4 digits)",Consolidation code,Last avail. year,...,Latitude,Longitude,Website address,"NAICS 2017, core code (4 digits)","NAICS 2017, core code - description",Size classification,Type of entity,Ethnic minority owned indicator (in US),Woman owned indicator (in US),Study Area
26791,INDELICATO INC,No,No,No,No,No,US,5610,LF,2019.0,...,"37° 46' 49.5"" N","122° 28' 43.8"" W",www.gasparespizza.com,7223,Special Food Services,Small company,Corporate,No,No,San Francisco


,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,BorrowerZip,LoanStatusDate,...,Gender,Veteran,NonProfit,zip_code_5,Minority,YearApproved,NAICS_6,LongName,IndustrySubsector,NAICS_4
206,6336148401,02/10/2021,912.0,PPS,INDELICATO INC.,5546 Geary Blvd,San Francisco,CA,94121-2209,NaN,...,Unanswered,Unanswered,NaN,94121,Yes,2021,722511.0,Full-Service Restaurants,Restaurants and Other Eating Places,7225.0


In [8]:
# TODO: STANDARDIZE ADDRESSES ACROSS BOTH DATA FRAME
display(orbis_df[orbis_df['Company name Latin alphabet'] == 'INDELICATO INC']['Address line 1\nLatin Alphabet'] )
display(ppp_df[ppp_df['BorrowerName'] == 'INDELICATO INC.']['BorrowerAddress'])

26791    5546 GEARY BLVD
Name: Address line 1\nLatin Alphabet, dtype: object

206    5546 Geary Blvd
Name: BorrowerAddress, dtype: object

In [16]:
# NAICS Code doesn't match
display(orbis_df[orbis_df['Company name Latin alphabet'] == 'INDELICATO INC']['NAICS 2017, core code (4 digits)'])
# TODO: Change NAICS columns' data types to int
display(ppp_df[ppp_df['BorrowerName'] == 'INDELICATO INC.']['NAICS_4'])

26791    7223
Name: NAICS 2017, core code (4 digits), dtype: int64

206    7225.0
Name: NAICS_4, dtype: float64

In [17]:
# Naive Solution - Get Rid of Punctuation
print('INDELICATO INC vs. INDELICATO INC.')
print('is it a match?', 'INDELICATO INC' == 'INDELICATO INC.')

# Use regex to strip punctuation (not characters or spaces)
print('after modifying, is it a match?', 'INDELICATO INC' == re.sub(r'[^\w\s]','','INDELICATO INC.'))

INDELICATO INC vs. INDELICATO INC.
is it a match? False
after modifying, is it a match? True


### 2. No Company Abbrv In One Name (INC. vs. no INC.)

Orbis:'RISTORANTE BUON GUSTO' <br>
PPP: 'RISTORANTE BUON GUSTO, INC' <br>

Solution: Remove all company abbreviations - using cleanco library <br>

Library: https://github.com/psolin/cleanco

Install library by running `pip install cleanco` in terminal

In [18]:
# No ', INC' like in the PPP Loan Data
display(orbis_df[orbis_df['Company name Latin alphabet'] == 'RISTORANTE BUON GUSTO'])
display(ppp_df[ppp_df['BorrowerName'] == 'RISTORANTE BUON GUSTO, INC'])

,Company name Latin alphabet,Inactive,Quoted,Branch,OwnData,Woco,Country ISO code,"NACE Rev. 2, core code (4 digits)",Consolidation code,Last avail. year,...,Longitude,Website address,"NAICS 2017, core code (4 digits)","NAICS 2017, core code - description",Size classification,Type of entity,Ethnic minority owned indicator (in US),Woman owned indicator (in US),Study Area,Company Name - Clean
90009,RISTORANTE BUON GUSTO,No,No,No,No,No,US,4725,LF,2019.0,...,"122° 24' 33.2"" W",www.ristorantebuongusto.com,4453,"Beer, Wine, and Liquor Stores",Medium sized company,Corporate,No,No,South San Francisco,RISTORANTE BUON GUSTO


,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,BorrowerZip,LoanStatusDate,...,Gender,Veteran,NonProfit,zip_code_5,Minority,YearApproved,NAICS_6,LongName,IndustrySubsector,NAICS_4
190,6685727803,06/02/2020,912.0,PPP,"RISTORANTE BUON GUSTO, INC",224 Grand Avenue,South San Francisco,CA,94080,04/29/2021,...,Male Owned,Non-Veteran,NaN,94080,Yes,2020,722511.0,Full-Service Restaurants,Restaurants and Other Eating Places,7225.0


## Solution: Finding PPP Loan Businesses In Orbis Database -- Using Cleanco

In [9]:
# Source: https://jakevdp.github.io/blog/2017/12/05/installing-python-packages-from-jupyter/
# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install cleanco

In [7]:
# Sample
from cleanco import cleanco
bus_name = 'RISTORANTE BUON GUSTO, INC'
bus_name1 = 'INDELICATO INC.'
bus_name2 = 'INDELICATO INC'

# Pass string into cleanco object
x = cleanco(bus_name)
y = cleanco(bus_name1)
z = cleanco(bus_name2)

# Use function .clean_name() for the business name without company abbreviations 
x.clean_name(), y.clean_name(), z.clean_name()

('RISTORANTE BUON GUSTO', 'INDELICATO', 'INDELICATO')

In [8]:
# Clean Orbis Name Column
orbis_df['Company Name - Clean'] =orbis_df['Company name Latin alphabet'].apply(lambda x: cleanco(x).clean_name())
orbis_df.head()

,Company name Latin alphabet,Inactive,Quoted,Branch,OwnData,Woco,Country ISO code,"NACE Rev. 2, core code (4 digits)",Consolidation code,Last avail. year,...,Longitude,Website address,"NAICS 2017, core code (4 digits)","NAICS 2017, core code - description",Size classification,Type of entity,Ethnic minority owned indicator (in US),Woman owned indicator (in US),Study Area,Company Name - Clean
0,CALIFORNIA NURSES ASSOCIATION,No,No,Yes,No,No,US,9412,LF,2019.0,...,"122° 16' 1.2"" W",www.calnursesfoundation.org,8139,"Business, Professional, Labor, Political, and ...",Very large company,Corporate,No,No,Oakland,CALIFORNIA NURSES ASSOCIATION
1,HONDA CARS OF CORONA,No,No,No,No,No,US,4519,LF,2019.0,...,"122° 15' 37.2"" W",www.hondacarsofcorona.com,4411,Automobile Dealers,Large company,Corporate,No,No,Oakland,HONDA CARS OF CORONA
2,SIMI MANAGEMENT CORP,No,No,No,No,No,US,4519,LF,2019.0,...,"122° 15' 48.6"" W",NaN,4411,Automobile Dealers,Large company,Corporate,No,No,Oakland,SIMI MANAGEMENT
3,ECONOMY LUMBER CO OF OAKLAND,No,No,No,No,No,US,4778,LF,2019.0,...,"122° 13' 10.9"" W",www.economylumberco.com,4441,Building Material and Supplies Dealers,Large company,Corporate,No,No,Oakland,ECONOMY LUMBER CO OF OAKLAND
4,BROADWAY FORD,No,No,No,No,No,US,4519,LF,2008.0,...,"122° 15' 53.5"" W",www.mybroadwayford.com,4411,Automobile Dealers,Large company,Corporate,No,No,Oakland,BROADWAY FORD


In [9]:
len(orbis_df)

92300

In [14]:
ppp_df['BorrowerName']

0         MARKET AUTO TRUCK COLLISION CORPORATION
1                       GOLD RUSH KETTLE KORN LLC
2                                KALIBER LABS INC
3                       ROSS MCDONALD COMPANY INC
4                          GATES EISENHART DAWSON
                           ...                   
249007            LARGE'S METAL FABRICATION, INC.
249008                        A.L. COMPANEROS INC
249009                               ONERENT INC.
249010              CANADIAN-AMERICAN OIL COMPANY
249011            SANTA ROSA SEAFOOD RETAIL, INC.
Name: BorrowerName, Length: 249012, dtype: object

In [10]:
# Clean PPP Name Column
ppp_df['BorrowerName - Clean'] = ppp_df['BorrowerName'].apply(str).apply(lambda x: cleanco(x).clean_name())
ppp_df.head()

,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,BorrowerZip,LoanStatusDate,...,ForgivenessAmount,ForgivenessDate,zip_code_5,Minority,YearApproved,NAICS_6,LongName,IndustrySubsector,NAICS_4,BorrowerName - Clean
0,8585717704,05/01/2020,459.0,PPP,MARKET AUTO TRUCK COLLISION CORPORATION,140 SAN JOSE AVE,SAN JOSE,AL,95125,0,...,15281.29,05/13/2021,95125,Unanswered,2020,811121.0,"Automotive Body, Paint, and Interior Repair an...",Automotive Repair and Maintenance,8111.0,MARKET AUTO TRUCK COLLISION
1,1464468601,03/13/2021,912.0,PPS,GOLD RUSH KETTLE KORN LLC,4690 E 2nd St Ste 9,Benicia,CA,94510-1008,0,...,0.00,NaN,94510,No,2021,722330.0,Mobile Food Services,Special Food Services,7223.0,GOLD RUSH KETTLE KORN
2,2663308509,02/22/2021,912.0,PPS,KALIBER LABS INC,188 King St Unit 307,San Francisco,CA,94107-4903,0,...,0.00,NaN,94107,No,2021,541511.0,Custom Computer Programming Services,Computer Systems Design and Related Services,5415.0,KALIBER LABS
3,4137348607,03/18/2021,912.0,PPS,ROSS MCDONALD COMPANY INC,1154 Stealth St,Livermore,CA,94551-9300,0,...,0.00,NaN,94551,Unanswered,2021,541420.0,Industrial Design Services,Specialized Design Services,5414.0,ROSS MCDONALD COMPANY
4,4315748602,03/18/2021,912.0,PPS,GATES EISENHART DAWSON,125 S Market St Ste 1200,San Jose,CA,95113-2288,0,...,0.00,NaN,95113,Unanswered,2021,541110.0,Offices of Lawyers,Legal Services,5411.0,GATES EISENHART DAWSON


In [11]:
len(ppp_df)

270064

In [12]:
# Find businesses in Orbis databases
ppp_from_orbis = ppp_df[ppp_df['BorrowerName - Clean'].isin(orbis_df['Company Name - Clean'])]
ppp_from_orbis.head()

,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,BorrowerZip,LoanStatusDate,...,ForgivenessAmount,ForgivenessDate,zip_code_5,Minority,YearApproved,NAICS_6,LongName,IndustrySubsector,NAICS_4,BorrowerName - Clean
19,9412638500,03/12/2021,912.0,PPS,PACIFICA PIZZA,915 1st St,Benicia,CA,94510-3217,0,...,0.00,NaN,94510,No,2021,722513.0,Limited-Service Restaurants,Restaurants and Other Eating Places,7225.0,PACIFICA PIZZA
72,6440528506,03/03/2021,912.0,PPS,435 EL CAMINO REAL INC,435 El Camino Real,South San Francisco,CA,94080-4345,0,...,0.00,NaN,94080,Unanswered,2021,722511.0,Full-Service Restaurants,Restaurants and Other Eating Places,7225.0,435 EL CAMINO REAL
77,7041187708,05/01/2020,912.0,PPP,"SPORTS PAGE, INC",1431 PLYMOUTH ST,MOUNTAIN VIEW,CA,94043-1227,03/17/2021,...,151052.16,02/12/2021,94043,Unanswered,2020,722410.0,Drinking Places (Alcoholic Beverages),Drinking Places (Alcoholic Beverages),7224.0,SPORTS PAGE
100,8712608401,02/13/2021,912.0,PPS,E & L RESTAURANT INC,1183 S De Anza Blvd Ste 30,San Jose,CA,95129-3659,0,...,0.00,NaN,95129,Yes,2021,722511.0,Full-Service Restaurants,Restaurants and Other Eating Places,7225.0,E & L RESTAURANT
102,5213408402,02/08/2021,912.0,PPS,SIMCO COFFEE LLC,PIER 39 SUITE 213,SAN FRANCISCO,CA,94133,0,...,0.00,NaN,94133,Unanswered,2021,722515.0,Snack and Nonalcoholic Beverage Bars,Restaurants and Other Eating Places,7225.0,SIMCO COFFEE


In [13]:
# Num of matches
len(ppp_from_orbis)

9599

In [14]:
ppp_from_orbis.to_csv(d+'/ppp-loan-data/out/orbis_ppp_all_bay.csv')